# What
Our code is a mess. This is just to not have to run everything all the time. Allows us to focus on the data we have instead of trying to get even more messy data.

This should be used as a default and copies should be done to build on top. The idea of this is that it's a basic AF readers so that actual analysis can be done.

In [240]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import statistics
import math
import plotly.express as px
import plotly.graph_objects as go

# networkx
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

# For semantic similarity
from urllib.parse import unquote
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Python functions in .py file to read data
import data_readers

import warnings
from tqdm import TqdmWarning
warnings.filterwarnings('ignore', category=TqdmWarning)

# Our custom plotting functions
import plot_functions

In [241]:
# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# DF of all articles and their categories
categories = data_readers.read_categories()

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

source_target_info = data_readers.source_target_paths_information()

In [242]:
# This is a last step, of making these three datasets have consistent column names and order
# as well as guaranteeing that the index order is conistent with both elements
# Even though the boolean column is weird, it's a boolean so that's all good

# FUCK IT. We don't touch the decoded version, that's too painful!

col_names = ['source', 'target', 'reached', 'length', 'visited']

import ast

carlos_df = pd.read_csv('machine_outputs/data_carlos.csv')
carlos_df.columns = col_names

carlos_df['visited'] = carlos_df['visited'].str.strip('\\').str.strip('{}')
carlos_df['visited'] = carlos_df['visited'].apply(lambda x: ast.literal_eval(x))

semantic_rank_df = pd.read_csv('machine_outputs/carolOutput.csv')
semantic_rank_df.columns = col_names

semantic_rank_df['visited'] = semantic_rank_df['visited'].str.strip('\\').str.strip('{}')
semantic_rank_df['visited'] = semantic_rank_df['visited'].apply(lambda x: ast.literal_eval(x))

landmark_df = pd.read_csv('machine_outputs/landmark_method_results.csv')

# Landmark_df is the only one with substantive change
landmark_df.drop('Unnamed: 0', axis=1, inplace=True)
landmark_df['reached'] = True
landmark_df.columns = ['source', 'target', 'length', 'visited', 'reached']
landmark_df = landmark_df[col_names]
# Due to the way length was counted here, I need to update to increase by one
landmark_df['length'] = landmark_df['length'] + 1

landmark_df['visited'] = landmark_df['visited'].str.strip('\\').str.strip('{}')
landmark_df['visited'] = landmark_df['visited'].apply(lambda x: ast.literal_eval(x))
landmark_df = landmark_df.sort_values('source')

In [243]:
finished_paths = finished_paths[['first_article', 'last_article', 'path_length', 'path']]
finished_paths = finished_paths.rename(columns={'first_article':'source', 'last_article':'target', 'path_length':'length','path':'visted'})

Modify given datasets

In [244]:
unfinished_paths['source'] = unfinished_paths['path'].str.split(';').apply(lambda x: x[0])
unfinished_paths = unfinished_paths[['source', 'target', 'path', 'type']]


In [245]:
def add_category_of_target_article(df_to_add_category_to) -> pd.DataFrame:
    categories[categories['article'].duplicated()]
    categories['article'] = categories['article'].drop_duplicates()
    sub_categories = categories['categories'].str[8:].str.split('.')
    category_depth_1 = sub_categories.apply(lambda x: x[0])
    categories['category'] = category_depth_1

    new_df = pd.merge(df_to_add_category_to, categories, left_on = 'target', right_on= 'article', how = 'left')
    new_df = new_df.drop(columns = ['article', 'categories'])
    return new_df

finished_paths = add_category_of_target_article(finished_paths)
landmark_df = add_category_of_target_article(landmark_df)
semantic_rank_df = add_category_of_target_article(semantic_rank_df)
carlos_df = add_category_of_target_article(carlos_df)

When are human paths shorter than the machine paths?

In [246]:
def combine_human_and_machine(df_human, df_machine) -> pd.DataFrame:
    df = pd.merge(df_human, df_machine, how = 'inner', left_on= ['first_article', 'last_article'], right_on = ['Source', 'Target'])
    df.rename(columns= {'path_length': 'man_len', 'len_shortest_path': 'machine_len'}, inplace=True)
    return df

In [251]:
a = finished_paths.groupby('category')['length'].mean()
b = landmark_df.groupby('category')['length'].mean()
c = semantic_rank_df.groupby('category')['length'].mean()
d = carlos_df.groupby('category')['length'].mean()

avg_lengths_df = pd.DataFrame((a,b,c, d)).T
avg_lengths_df.columns = ['human', 'landmark', 'semantic_rank', 'carlos']
avg_lengths_df = avg_lengths_df.sort_values('human')

plot_functions.barplot_groups(avg_lengths_df, file_name= 'avg_lengths', title='Average Finished Path Length by Category of Machine vs Man', barmode='group')

Can machine do some of the unfinished paths that man couldn’t?


In [313]:
def merge_man_and_machine_df(human_df, machine_df) -> pd.DataFrame:
    return pd.merge(human_df, machine_df, how = 'inner', on = ['source', 'target'])

In [285]:
merge_man_and_machine_df(unfinished_paths, landmark_df)['reached'].value_counts()
print('Out of the unfinished human games that the Landmark algorithm attempted, the Landmark algorithm was able to complete 100% of the games')

human_unfinished_vs_semantic_rank = merge_man_and_machine_df(unfinished_paths, semantic_rank_df)
semantic_rank_percent_completed = human_unfinished_vs_semantic_rank['reached'].value_counts()[0] / human_unfinished_vs_semantic_rank.shape[0]
print('Out of the unfinished human games that the Semantic Rank algorithm attempted, it was able to complete', np.round(semantic_rank_percent_completed,10)*100, '% of the games.')

Out of the unfinished human games that the Landmark algorithm attempted, the Landmark algorithm was able to complete 100% of the games
Out of the unfinished human games that the Semantic Rank algorithm attempted, it was able to complete 92.70083501 % of the games.


In [315]:
unfinished_paths_restart = unfinished_paths[unfinished_paths['type'] == 'restart']

merge_man_and_machine_df(unfinished_paths_restart, landmark_df)['reached'].value_counts()
print('Next, we filter based on ')
print('Out of the unfinished human games that the Landmark algorithm attempted, the Landmark algorithm was able to complete 100% of the games')

human_unfinished_restart_vs_semantic_rank = merge_man_and_machine_df(unfinished_paths_restart, semantic_rank_df)
semantic_rank_percent_completed = human_unfinished_restart_vs_semantic_rank['reached'].value_counts()[0] / human_unfinished_restart_vs_semantic_rank.shape[0]
print('Out of the unfinished human games that the Semantic Rank algorithm attempted, it was able to complete', np.round(semantic_rank_percent_completed, 10)*100, '% of the games.')

Out of the unfinished human games that the Landmark algorithm attempted, the Landmark algorithm was able to complete 100% of the games
Out of the unfinished human games that the Semantic Rank algorithm attempted, it was able to complete 92.92582418 % of the games.
